# **Spectral Indices (Year 2025)**

This study evaluates the land surface characteristics of the selected study area using Landsat 9 imagery for the year 2025. Five spectral indices were computed—NDVI, NDWI, SAVI, NDBI, and EVI—to analyze vegetation health, surface water distribution, soil–vegetation interaction, and built-up expansion. These indices transform raw satellite reflectance values into meaningful environmental indicators, enabling a clearer understanding of spatial patterns within the region.

In [ ]:
import ee

In [ ]:
import geemap

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='tanu-p-483509')

In [ ]:
import geemap as map
map

<module 'geemap' from '/usr/local/lib/python3.12/dist-packages/geemap/__init__.py'>

In [ ]:
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
Tehsil = ee.FeatureCollection("projects/tanu-p-483509/assets/Tehsil_Boundary")
map.addLayer(Tehsil,{},"Tehsil")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
Landsat9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
L9_Optical = Landsat9.select("SR_B1","SR_B2","SR_B3","SR_B4","SR_B5","SR_B6")
vis_params = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'gamma': 1.0, 'min': 8209.4, 'max': 12778.0}
vis_params9 = {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'gamma': 1.0, 'min': 8456.6, 'max': 16724.0}
Tehsil9 = L9_Optical.filterBounds(Tehsil).filterDate("2025-02-01","2025-06-15").filterMetadata("CLOUD_COVER","less_than",5).mean().clip(Tehsil)
map.addLayer(Tehsil9,vis_params,"Tehsil9_2025_TCC")
map.addLayer(Tehsil9,vis_params9,"Tehsil9_2025_FCC")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
def scale_landsat9(image):
  optical = image.select(["SR_B1","SR_B2","SR_B3","SR_B4","SR_B5","SR_B6"]).multiply(0.0000275).add(-0.2)
  return image.addBands(optical,overwrite=True)

In [ ]:
scaled_L9 = scale_landsat9(Tehsil9)

In [ ]:
Green = scaled_L9.select("SR_B3")
Moisture = scaled_L9.select("SR_B6")
NIR = scaled_L9.select("SR_B5")
Red = scaled_L9.select("SR_B4")

# **NDVI (Normalized Difference Vegetation Index)**

NDVI is widely used to assess vegetation health and density. It is calculated using near-infrared (NIR) and red reflectance values. Healthy vegetation reflects more NIR and absorbs red light, resulting in higher NDVI values closer to +1. Conversely, barren land, built-up areas, and water bodies show low or negative NDVI values.

The NDVI map for 2025 indicates that high vegetation density is concentrated in agricultural lands and forested patches. These areas appear with strong positive values, suggesting active photosynthetic activity. Moderate NDVI values are observed in mixed land cover zones where vegetation is sparse or seasonal. Low NDVI values are primarily associated with urban areas, exposed soil, and water bodies. The spatial distribution clearly differentiates vegetated regions from non-vegetated surfaces, reflecting the ecological structure of the study area.

In [ ]:
NDVI25 = NIR.subtract(Red).divide(NIR.add(Red)).rename("NDVI25")
vis_NDVI = {'bands': ['NDVI25'], 'palette': ['#f7fcf5', '#c7e9c0', '#74c476', '#238b45', '#00441b'], 'min': -0.5473063140567523, 'max': 0.15477716609205924}
map.addLayer(NDVI25,vis_NDVI,"Tehsil_NDVI25")
map.centerObject(Tehsil,11)
map

Map(center=[18.853649630989338, 73.16268651463666], controls=(WidgetControl(options=['position', 'transparent_…

# **SAVI (Soil Adjusted Vegetation Index)**

SAVI modifies the NDVI formula by incorporating a soil brightness correction factor. This makes it particularly useful in areas where vegetation cover is sparse and soil reflectance influences the spectral signal.

In the 2025 SAVI map, vegetation patterns appear more refined in semi-arid or partially vegetated regions. Compared to NDVI, SAVI reduces the overestimation of vegetation in areas where exposed soil is present. Moderate SAVI values indicate transitional zones between cultivated land and barren surfaces. The results suggest that soil background plays a noticeable role in influencing vegetation signals in certain parts of the study area. Therefore, SAVI provides a more reliable representation of vegetation cover in mixed landscapes.

In [ ]:
SAVI = (NIR.subtract(Red).divide(NIR.add(Red).add(0.5))).multiply(1.5).rename("SAVI")
vis_savi = {'bands': ['SAVI'], 'palette': ['#543005', '#f5f5f5', '#003c30'], 'min': -0.08312042740358679, 'max': 0.07202014883650243}
map.addLayer(SAVI,vis_savi,"Tehsil_SAVI25")
map

Map(center=[18.853649630989338, 73.16268651463666], controls=(WidgetControl(options=['position', 'transparent_…

# **NDWI (Normalized Difference Water Index)**

NDWI is used to identify surface water features such as rivers, lakes, ponds, and reservoirs. It enhances water reflectance while suppressing vegetation and soil signals. Positive NDWI values typically represent water bodies.

The NDWI map for 2025 successfully highlights major water features within the study area. River channels and reservoir zones appear distinctly with higher index values. Surrounding agricultural and urban regions show negative values, indicating absence of surface water. The clear delineation of water bodies demonstrates the effectiveness of NDWI in hydrological mapping. Spatial patterns suggest that water distribution is localized and concentrated along natural drainage systems, with limited spread into surrounding land areas.

In [ ]:
NDWI = Green.subtract(NIR).divide(Green.add(NIR)).rename("NDWI")
vis_NDWI = {'bands': ['NDWI'], 'palette': ['#f7fbff', '#6baed6', '#08306b'], 'min': -0.43802902682261624, 'max': 0.7145446977755173}
map.addLayer(NDWI,vis_NDWI,"Tehsil_NDWI25")
map

Map(center=[18.853649630989338, 73.16268651463666], controls=(WidgetControl(options=['position', 'transparent_…

# **NDMI (Normalized Difference Moisture Index)**

NDMI measures vegetation moisture content using NIR and SWIR bands. Higher NDMI values indicate healthy vegetation with sufficient moisture, while low or negative values represent dry vegetation, barren land, or built-up areas.

The NDMI map for 2025 shows higher moisture levels in vegetated and irrigated agricultural regions. Lower values are observed in urban and exposed soil areas. This indicates spatial variation in surface moisture conditions across the study area. NDMI helps identify moisture stress and supports agricultural and environmental monitoring.

In [ ]:
NDMI = NIR.subtract(Moisture).divide(NIR.add(Moisture)).rename("NDMI")
vis_NDMI = {'bands': ['NDMI'], 'palette': ['#f7fcf0', '#7bccc4', '#084081'], 'min': 0.059958009052027944, 'max': 0.4071485834429973}
map.addLayer(NDMI,vis_NDMI,"Tehsil_NDMI25")
map

Map(center=[18.853649630989338, 73.16268651463666], controls=(WidgetControl(options=['position', 'transparent_…

# **NDBI (Normalized Difference Built-up Index)**

NDBI is specifically designed to identify built-up or urban areas using shortwave infrared (SWIR) and near-infrared (NIR) bands. Built-up surfaces reflect more SWIR radiation compared to vegetation.

The NDBI map for 2025 clearly identifies urban settlements and infrastructure zones. Positive values correspond to developed land areas, while negative values represent vegetation and water bodies. The spatial distribution of high NDBI values indicates concentrated urban development within specific regions. These built-up clusters contrast sharply with surrounding agricultural or vegetated areas. The index effectively captures the extent of anthropogenic land transformation and highlights zones of urban expansion.

In [ ]:
NDBI = Moisture.subtract(NIR).divide(Moisture.add(NIR)).rename("NDBI")
vis_NDBI = {'bands': ['NDBI'], 'palette': ['#f7f4f9', '#df65b0', '#67001f'], 'min': -0.4071485834429973, 'max': -0.059958009052027944}
map.addLayer(NDBI,vis_NDBI,"Tehsil_NDBI25")
map

Map(bottom=234477.0, center=[18.853649630989338, 73.16268651463666], controls=(WidgetControl(options=['positio…

In [ ]:
NDVI25_fc = NDVI25.sample(region=Tehsil, scale=30)
SAVI_fc = SAVI.sample(region=Tehsil, scale=30)
NDWI_fc = NDWI.sample(region=Tehsil, scale=30)
NDMI_fc = NDMI.sample(region=Tehsil, scale=30)
NDBI_fc = NDBI.sample(region=Tehsil, scale=30)

In [ ]:
geemap.ee_export_vector_to_drive(NDVI25_fc, "NDVI_2025.geojson", folder='Raster_Analysis', fileFormat='geojson')
geemap.ee_export_vector_to_drive(SAVI_fc, "SAVI_2025.geojson", folder='Raster_Analysis', fileFormat='geojson')
geemap.ee_export_vector_to_drive(NDWI_fc, "NDWI_2025.geojson", folder='Raster_Analysis', fileFormat='geojson')
geemap.ee_export_vector_to_drive(NDMI_fc, "NDMI_2025.geojson", folder='Raster_Analysis', fileFormat='geojson')
geemap.ee_export_vector_to_drive(NDBI_fc, "NDBI_2025.geojson", folder='Raster_Analysis', fileFormat='geojson')

Exporting NDVI_2025.geojson... Please check the Task Manager from the JavaScript Code Editor.
Exporting SAVI_2025.geojson... Please check the Task Manager from the JavaScript Code Editor.
Exporting NDWI_2025.geojson... Please check the Task Manager from the JavaScript Code Editor.
Exporting NDMI_2025.geojson... Please check the Task Manager from the JavaScript Code Editor.
Exporting NDBI_2025.geojson... Please check the Task Manager from the JavaScript Code Editor.


In [ ]:
def export_index_csv(index_image, index_name):

    samples = index_image.sample(
        region=Tehsil.geometry(),
        scale=30,
        geometries=True
    )
    samples = samples.map(lambda feature: feature.set({
        'longitude': feature.geometry().coordinates().get(0),
        'latitude': feature.geometry().coordinates().get(1)
    }))

    task = ee.batch.Export.table.toDrive(
        collection=samples,
        description=f'{"CSV"}_2025_CSV',
        folder='Raster_Analysis',
        fileNamePrefix=f'{index_name}_2025',
        fileFormat='CSV'
    )

    task.start()


In [ ]:
export_index_csv(NDVI25, "NDVI")

In [ ]:
export_index_csv(NDWI, "NDWI")

In [ ]:
export_index_csv(SAVI, "SAVI")

In [ ]:
export_index_csv(NDBI, "NDBI")

In [ ]:
export_index_csv(NDMI, "NDMI")

In [4]:
import pandas as pd

In [7]:
ndvi_df = pd.read_csv("/content/drive/MyDrive/Raster_Analysis/NDVI_2025.csv")
ndvi_df.to_html("ndvi.html", index=False)

In [ ]:
ndwi_df = pd.read_csv("/content/drive/MyDrive/Raster_Analysis/NDWI_2025.csv")
ndwi_df.to_html("ndwi.html", index=False)
savi_df = pd.read_csv("/content/drive/MyDrive/Raster_Analysis/SAVI_2025.csv")
savi_df.to_html("savi.html", index=False)
ndbi_df = pd.read_csv("/content/drive/MyDrive/Raster_Analysis/NDBI_2025.csv")
ndbi_df.to_html("ndbi.html", index=False)
ndmi_df = pd.read_csv("/content/drive/MyDrive/Raster_Analysis/NDMI_2025.csv")
ndmi_df.to_html("ndmi.html", index=False)